#**Listar arquivos da pasta Carteiras**

In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
diretorio='/content/drive/MyDrive/Carteiras/'

In [4]:
os.listdir(diretorio)

['SALDOS_C.TXT',
 'TB_DIST_HISOFI.xlsx',
 'TABELA_ESSENCIAL_HISOFI.xlsx',
 'ESC_565.CSV',
 'ESC_ESSEN_565.CSV',
 'Asignacion20230922.xlsx',
 'Rep_CarteraApha_22092023_1207.csv',
 'ASIG CARTERA SET - HISOFI - 22.09.2023.xlsx',
 '1691077668699template_sofi.xlsx']

In [5]:
print(len(os.listdir(diretorio)))

9


# **Triagem dos Arquivos baixados**


## Inicialização

In [6]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Apagar COLLECTI.ZIP se existir

In [8]:
mydrive='/content/drive/MyDrive/'
dir_carteiras=mydrive+'Carteiras/'
#Remover COLLECTI.ZIP
try:
  os.remove(mydrive+'COLLECTI.ZIP')
except:
  print('Arquivo COLLECTI.ZIP não existente')
#Remover SALDOS_C.ZIP
try:
  os.remove(mydrive+'SALDOS_C.ZIP')
except:
  print('Arquivo SALDOS_C.ZIP não existente')

Arquivo COLLECTI.ZIP não existente
Arquivo SALDOS_C.ZIP não existente


##Importando Tabela de Diretórios

In [9]:
xlsx_diretorios=pd.read_excel(mydrive+'Processamentos/Tratamento/auxiliary/Diretorios.xlsx',sheet_name='Arquivos',dtype=object)
nomes=np.array(xlsx_diretorios['Nome']).astype(str)
carteiras=np.array(xlsx_diretorios['Carteira'])
diretorios=np.array(xlsx_diretorios['Diretório'])
print(nomes)

['Asignacion' '1800' '1801' 'TABELA_ESSENCIAL_HISOFI' 'ESC_ESSEN_565'
 'ASIG CARTERA' 'Rep_CarteraApha' 'TB_DIST' 'ESC_565' 'TB_BATIMENTO'
 'COLLECTI' 'Plantón' 'Descuentos' 'natura-mex-padrao'
 'natura-peru-exclusiva' 'ESPECIAL' 'natura-mex-tardia'
 'pronto-carteira-1' 'porto-essencial' 'porto-tradicional' 'SALDOS_C'
 'Late Stage' 'Descuentos' 'template_sofi']


##Checando quantidade de arquivos

In [10]:
num_arquivos=len(os.listdir(dir_carteiras))
if num_arquivos<8:
  print('Tem somente',num_arquivos,'arquivos de carteira, deveria ter mais de 8')
  sys.exit()

##Movendo arquivos para as pastas corretas

In [11]:
rodar=[]
for arquivo in os.listdir(dir_carteiras):
  print(arquivo)
  for i in range(len(nomes)):
    if arquivo.find(nomes[i])>=0:
      rodar.append(carteiras[i])
      os.makedirs(diretorios[i],exist_ok=True)
      os.rename(dir_carteiras+arquivo,diretorios[i]+arquivo)
      continue

SALDOS_C.TXT
TB_DIST_HISOFI.xlsx
TABELA_ESSENCIAL_HISOFI.xlsx
ESC_565.CSV
ESC_ESSEN_565.CSV
Asignacion20230922.xlsx
Rep_CarteraApha_22092023_1207.csv
ASIG CARTERA SET - HISOFI - 22.09.2023.xlsx
1691077668699template_sofi.xlsx


##Carteiras que tiveram arquivos movidos

In [12]:
rodar=np.array(rodar)
rodar=np.unique(rodar)
rodar=np.sort(rodar)
print(list(rodar),len(rodar))

['Avon PE', 'Natura MEX', 'Natura PE', 'Porto Batimento', 'Porto Essencial', 'Pronto', 'Risel'] 7


# **Tratamento NATURA PE EXCLUSIVA**

## Inicialização

In [13]:
tem_especial=True
mover=True

In [14]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
carteira='Natura PE'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
ingestion=diretorio+'ingestion/'
planton=diretorio+'Plantón/'
processedfiles=diretorio+'processedfiles/'
dir_especial=diretorio+'Especial/'
dir_num_errados=diretorio+'Números Inválidos/'
os.makedirs(dir_num_errados,exist_ok=True)

today=datetime.date.today()
#Nome do arquivo de saída com caminho
csv_saida=output+'early_natura-peru-exclusiva_'+str(today.strftime('%d-%m-%Y'))+'.csv'
#Arquivo com números inválidos
errados_saida=dir_num_errados+'numeros-errados_natura-pe-exclusiva_'+str(today)+'.xlsx'

In [17]:
#Esperando ler 1
num_arquivo=1

while len(os.listdir(ingestion))!=num_arquivo:
  time.sleep(5)
print(os.listdir(ingestion))

['ASIG CARTERA SET - HISOFI - 22.09.2023.xlsx']


In [18]:
#Identificando as planilhas EXCLUSIVA e ESPECIAL
for arquivo in os.listdir(ingestion):
    if arquivo.find('ASIG')>=0:
      nome_exclusiva=arquivo
      arquivo_exclusiva=ingestion+arquivo

#Identificando o arquivo mais recente da pasta Exclusiva
if tem_especial:
  df_arquivo_recente=pd.DataFrame({'Nome':os.listdir(dir_especial)})
  tempo_modif=np.empty(len(df_arquivo_recente))
  i=0
  for arquivo in os.listdir(dir_especial):
    tempo_modif[i]=os.path.getmtime(dir_especial+arquivo)
    i+=1
  df_arquivo_recente['Tempo de Modificacao']=tempo_modif
  df_arquivo_recente=df_arquivo_recente.sort_values(by='Tempo de Modificacao',ascending=False)
  df_arquivo_recente.index=range(len(df_arquivo_recente))
  print(df_arquivo_recente)
  nome_arquivo_especial=str(df_arquivo_recente['Nome'][0])
  arquivo_especial=dir_especial+nome_arquivo_especial
  print(arquivo_especial)

#Identificando o arquivo mais recente da pasta Plantón
df_arquivo_recente=pd.DataFrame({'Nome':os.listdir(planton)})
tempo_modif=np.empty(len(df_arquivo_recente))
i=0
for arquivo in os.listdir(planton):
  tempo_modif[i]=os.path.getmtime(planton+arquivo)
  i+=1
df_arquivo_recente['Tempo de Modificacao']=tempo_modif
df_arquivo_recente=df_arquivo_recente.sort_values(by='Tempo de Modificacao',ascending=False)
df_arquivo_recente.index=range(len(df_arquivo_recente))
print(df_arquivo_recente)
nome_arquivo_platon=str(df_arquivo_recente['Nome'][0])
arquivo_planton=planton+nome_arquivo_platon
print(arquivo_planton)

#csv_saida2=arquivo_exclusiva.replace('.xlsx',' - Tratado com Planilha.csv').replace(ingestion,output)
#xlsx_saida=arquivo_exclusiva.replace('.xlsx',' - Tratado.xlsx').replace(ingestion,output)

                                                Nome  Tempo de Modificacao
0  ASIG CARTERA JUL- HISOFI - ESPECIAL - 14.07.20...          1.690397e+09
/content/drive/MyDrive/Processamentos/Tratamento/Natura PE/Especial/ASIG CARTERA JUL- HISOFI - ESPECIAL - 14.07.2023.xlsx
                                                Nome  Tempo de Modificacao
0  CNs Plantón C12- 2023 - Actualizado 23 Agosto...          1.692904e+09
/content/drive/MyDrive/Processamentos/Tratamento/Natura PE/Plantón/CNs Plantón C12- 2023 - Actualizado 23 Agosto - HISOFI.xlsx


In [19]:
if tem_especial:
    print(arquivo_exclusiva,'\n',arquivo_especial,'\n',arquivo_planton)
else:
    print(arquivo_exclusiva,'\n',arquivo_planton)

/content/drive/MyDrive/Processamentos/Tratamento/Natura PE/ingestion/ASIG CARTERA SET - HISOFI - 22.09.2023.xlsx 
 /content/drive/MyDrive/Processamentos/Tratamento/Natura PE/Especial/ASIG CARTERA JUL- HISOFI - ESPECIAL - 14.07.2023.xlsx 
 /content/drive/MyDrive/Processamentos/Tratamento/Natura PE/Plantón/CNs Plantón C12- 2023 - Actualizado 23 Agosto - HISOFI.xlsx


In [20]:
#Carregando Planilhas
exclusiva=pd.read_excel(arquivo_exclusiva,dtype=object)
exclusiva['Planilha']='EXCLUSIVA'
print('Exclusiva carregado com sucesso!')
df_planton=pd.read_excel(arquivo_planton,dtype=object)
print('Plantón carregado com sucesso!')
if tem_especial:
    especial=pd.read_excel(arquivo_especial,dtype=object)
    print('Especial carregado com sucesso!')
    especial['Planilha']='ESPECIAL'
else:
    print('Carteira Especial não existente')

Exclusiva carregado com sucesso!
Plantón carregado com sucesso!
Especial carregado com sucesso!


## Início das Transformações

### Concatenar Exclusiva e Especial

In [21]:
#Concatenando Carteiras e criando cópias
if tem_especial:
    concatenar=pd.concat([exclusiva,especial],join='outer',ignore_index=True)
    cartera=concatenar.copy()
else:
    cartera=exclusiva.copy()
planton=df_planton.copy()

###Filtrar descontos da Planton por data

In [22]:
planton['Fecha Vigencia'] = pd.to_datetime(planton['Fecha Vigencia'])
filtro=planton['Fecha Vigencia']>=pd.Timestamp(today)
planton=planton[filtro]

### Capitalizar 'Nombre de la Persona'

In [23]:
#Capitalizando coluna de Nomes
cartera['Nombre de la Persona']=cartera['Nombre de la Persona'].str.title()
cartera['Nombre de la Persona']

0                     Melanie  Pereyra Salvador
1           Esther  Ubaldina Abramonte Castillo
2                    Almendra Nikole  Ruiz Pari
3               Rosa  Marina Mosquera Maldonado
4                     Rosa  Nerida Ulloa Vargas
                          ...                  
39699                  Diana Nicol  Pecho Rojas
39700             Luis Antonio  Arcentales Ruiz
39701          Josselin Xiomara  Falcon Riveros
39702           Kateryn Veronica  Cruz Guerrero
39703    Tatihana Loren Ruth  Sernaque Parrilla
Name: Nombre de la Persona, Length: 39704, dtype: object

### Tratando '% Descuento Saldo Original' para ficar que nem no excel em %

In [24]:
#Colocando % na coluna de Descontos e multiplicando por 100
cartera['% Descuento Saldo Original']=cartera['% Descuento Saldo Original'].astype(float)*100
cartera['% Descuento Saldo Original']=cartera['% Descuento Saldo Original'].astype(int)
cartera['% Descuento Saldo Original']=cartera['% Descuento Saldo Original'].apply(lambda x: str(x)+'%')
cartera['% Descuento Saldo Original']

0        0%
1        0%
2        0%
3        0%
4        0%
         ..
39699    0%
39700    0%
39701    0%
39702    0%
39703    0%
Name: % Descuento Saldo Original, Length: 39704, dtype: object

### Tratando datas da coluna 'Fecha nasc'

In [25]:
#Tratando a coluna de Fecha Nasc que tem datas em formato de número
coluna='Fecha nasc'
# cartera[coluna]=cartera[coluna].apply(lambda x:'' if str(x)=='-' else x)
data=np.array(cartera[coluna])
for i in range(len(data)):
  if str(data[i]).find('/')==-1 and str(data[i]).find('-')==-1:
      data[i]=str(pd.to_datetime('1900-01-01')+pd.Timedelta(days=data[i]-2))
cartera[coluna]=data
cartera[coluna]=cartera[coluna].apply(lambda x: str(x)[:10])

#Tratando as datas que estão com - para /
x=np.array(cartera[coluna])
print(x)
for i in range(len(x)):
  if x[i].find('/')==-1:
    x[i]=str(x[i])[-2:]+'/'+str(x[i])[5:7]+'/'+str(x[i])[:4]
cartera[coluna]=x
#Datas que são - viram -//- depois das operações acima, substituir essas datas por vazio
cartera[coluna]=cartera[coluna].str.replace('-//-','')
cartera[coluna]

['-' '20/01/1985' '05/10/2000' ... '2023-03-28' '2023-03-29' '2023-03-29']


0                  
1        20/01/1985
2        05/10/2000
3        23/11/1962
4        16/10/1973
            ...    
39699    27/03/2023
39700    28/03/2023
39701    28/03/2023
39702    29/03/2023
39703    29/03/2023
Name: Fecha nasc, Length: 39704, dtype: object

### Tirando o horário das colunas de data e convertendo pro formato brasileiro

In [26]:
#Tirando horário nas colunas de data
print(cartera.select_dtypes(include='datetime').columns.tolist())
for coluna in cartera.select_dtypes(include='datetime').columns.tolist():
    cartera[coluna]=cartera[coluna].apply(lambda x: str(x)[:10])
    cartera[coluna]=pd.to_datetime(cartera[coluna])
    cartera[coluna]=cartera[coluna].dt.strftime('%d/%m/%Y')

['Fecha Pedido', 'Fecha Vcto', 'Fecha Saldo', 'Fecha Saldo Corregido']


### Remover colunas 'Estructura Comercial','Estructura Comercial Padre'

In [27]:
#Apagando colunas de Estrutuctura Comercial
cartera=cartera.drop(['Estructura Comercial','Estructura Comercial Padre'],axis=1)

### Remover DNI nulos, zeros e - e preencher com 8 dígitos

In [28]:
#Tratando DNI nulo e hífens
print('Total de linhas antes:',len(cartera))
nulos=np.array(cartera['DNI'].notnull())
hifens=np.array(cartera['DNI']!='-')
filtro=nulos*hifens
print('Total de linhas diferente de - e nulo:',filtro.sum())
cartera=cartera[filtro]
#Formatando DNI com 8 dígitos
cartera['DNI']=cartera['DNI'].astype(object)
cartera['DNI']=cartera['DNI'].apply(lambda x: str(x).zfill(8))
#Removendo zeros
cartera=cartera[cartera['DNI']!='00000000']
print('Total de linhas depois de filtrar zeros:',len(cartera))
cartera.sort_values('DNI').head(3)

Total de linhas antes: 39704
Total de linhas diferente de - e nulo: 39322


<ipython-input-28-8ef5fa5ac3fd>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cartera['DNI']=cartera['DNI'].astype(object)
<ipython-input-28-8ef5fa5ac3fd>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cartera['DNI']=cartera['DNI'].apply(lambda x: str(x).zfill(8))


Total de linhas depois de filtrar zeros: 39282


,Orden del Pedidos,Código Persona,Cod Grupo,Nombre de la Persona,Título,No Pedido,No Factura,Fecha Pedido,Fecha Vcto,Fecha Saldo,...,Nombre de la Estructura Comercial Padre,DNI,Dirección,Teléfono Persona,Numero Celular,Correo Electrónico,TRAMO,AGENCIA,RIESGO,Planilha
6444,3,2888811,4607,Lina Socorro Paredes Manuyamadearajo,14833623,38832499,4560361,15/06/2023,14/07/2023,02/07/2023,...,HADAR,00006285,JR.VICTOR RAUL MZ.D 5 LT.1 AH. VICTOR RAUL HAY...,942192555,942192555,-,B) 31 - 60,HISOFI,ALTO,EXCLUSIVA
7475,4,2888811,4607,Lina Socorro Paredes Manuyamadearajo,14927533,38917521,4741519,28/06/2023,31/07/2023,02/07/2023,...,HADAR,00006285,JR.VICTOR RAUL MZ.D 5 LT.1 AH. VICTOR RAUL HAY...,942192555,942192555,-,B) 31 - 60,HISOFI,ALTO,EXCLUSIVA
1366,1,2925443,2998,Sonia Flor Loayza Rios Vda De Navarro,15034690,39048606,4966479,19/07/2023,17/08/2023,17/08/2023,...,MANANTIAL,00007536,JR. PADRE IRAZOLA MZ 9 LT 7B 0 AAAHH SAN JUAN ...,-,922117716,crisibn.25@gmail.om,A) 1 - 30,HISOFI,ALTO,EXCLUSIVA


### Adicionando coluna 'MONEDA'

In [29]:
#Criando coluna MONEDA
cartera['MONEDA']='PEN'

### Mesclar tabela Plantón

In [30]:
#Mesclando tabelas
codigo_importe=nome_arquivo_platon[13:16] #12:15 no script local
importe_a_pagar='Importe a pagar - Plantón Cobranza ' + codigo_importe     #'Importe a pagar - Plantón Cobranza ' + codigo_importe
planton=planton[['No Pedido',importe_a_pagar]]
#planton.columns=['No Pedido',importe_a_pagar]   #Renomear coluna de Saldo(Antigo Importe a pagar) porque já tem coluna Saldo na carteira
mesclado=cartera.merge(planton,on='No Pedido',how='left')

### Substituindo Monto Mínimo a Pagar pelo valor do Platón

In [31]:
#Substituindo Monto Mínimo a Pagar pelo valor do Platón
importe_nao_nulo=mesclado[importe_a_pagar].notnull()
importe_nao_nulo.sum()
valores=np.array(mesclado['Monto mínimo a pagar Campaña'])
valores[importe_nao_nulo]=mesclado[importe_nao_nulo][importe_a_pagar]
mesclado['Monto mínimo a pagar Campaña']=valores

###Criar coluna de Descuentos


In [32]:
mesclado['Descuentos'] = 0
mesclado.loc[importe_nao_nulo,'Descuentos']=1

### Apagar a coluna auxiliar 'Importe a pagar - Plantón Cobranza C10'

In [33]:
mesclado=mesclado.drop(importe_a_pagar,axis=1)

### Remover 'No Pedido' duplicados

In [34]:
#Remover Pedidos duplicados
print(len(mesclado))
mesclado=mesclado.drop_duplicates(subset='No Pedido')
print(len(mesclado))

39282
38909


### Arredondando e formatando colunas de números para 2 casas decimais

In [35]:
#Formatar números para 2 casas decimais
colunas=['Original', 'Saldo','Saldo Corregido','Monto mínimo a pagar Campaña','Interés/multa Acumulados (100% descuento)','Monto Descuento Saldo Original','Total descuentos']
for coluna in colunas:
    mesclado[coluna]=mesclado[coluna].astype(float)
    mesclado[coluna]=mesclado[coluna].round(2)
    mesclado[coluna]=mesclado[coluna].map('{:.2f}'.format)

###Tratamento dos números de telefone

In [36]:
#Fazendo coerce com Celular e Telefone
telefone=np.array(mesclado['Teléfono Persona'])
celular=np.array(mesclado['Numero Celular'])
celular_nao_nulo=mesclado['Numero Celular'].notnull()
telefone[celular_nao_nulo]=celular[celular_nao_nulo]
mesclado['Numero Celular']=telefone
#Tirando espaços
mesclado['Numero Celular']=mesclado['Numero Celular'].str.strip()
#Tratando números com mais de 9 dígitos e 8 dígitos
mesclado['Numero Celular']=mesclado['Numero Celular'].apply(lambda x: '9'+str(x) if len(str(x))==8 else (str(x)[-9:] if len(str(x))>9 else x))
#Apagar números que não começam com 9
mesclado['Numero Celular']=mesclado['Numero Celular'].apply(lambda x: '' if str(x)[0]!='9' else x)
#Apagar números com menos de 8 dígtios
filtro=np.char.str_len(np.char.array(mesclado['Numero Celular']))<8
#Exportar os números errados
mesclado.loc[:,'Numero Celular'][filtro].to_csv(errados_saida,index=False)
#Substituir por '' os números errados
mesclado.loc[:,'Numero Celular'][filtro]=''

<ipython-input-36-2bd38506ba13>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mesclado.loc[:,'Numero Celular'][filtro]=''


###Tratamento de Email

In [37]:
# Aplicando lower e strip na coluna de email
coluna_email = 'Correo Electrónico'
mesclado[coluna_email] = mesclado[coluna_email].str.lower().str.strip()
# Apagando emails não validados
mesclado[coluna_email].loc[mesclado[coluna_email].str.find('@') < 0] = ''
mesclado['num @'] = mesclado[coluna_email].str.len() - mesclado[coluna_email].str.replace('@', '').str.len()
filtro = mesclado['num @'] != 1
mesclado.loc[filtro, coluna_email] = ''
mesclado.drop('num @', axis=1, inplace=True)

<ipython-input-37-8d69745d573b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mesclado[coluna_email].loc[mesclado[coluna_email].str.find('@') < 0] = ''


###Tratar coluna de Nome

In [38]:
coluna_nomes=['Nombre de la Persona']
for coluna_nome in coluna_nomes:
  mesclado[coluna_nome]=mesclado[coluna_nome].str.strip().str.title()

## Exportando

In [39]:
#mesclado.to_csv(csv_saida2,index=False)
try:
  mesclado=mesclado.drop('Planilha',axis=1)
except:
  print('Coluna planilha já excluída')
#mesclado.to_excel(xlsx_saida,index=False)
mesclado.to_csv(csv_saida,index=False)

## Movendo Somente as Carteiras Exclusivas

In [40]:
if mover:
  arquivos=[nome_exclusiva]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento NATURA MEX PADRÃO**

In [41]:
mover=True

import pandas as pd
import numpy as np
import os
import time
import datetime

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Definindo Diretórios

In [43]:
carteira='Natura MEX'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
ingestion=diretorio+'ingestion/'
processedfiles=diretorio+'processedfiles/'
auxiliary=diretorio+'auxiliary/'
dir_num_errados=diretorio+'Números Inválidos/'
os.makedirs(dir_num_errados,exist_ok=True)
#Nome do arquivo de saída com caminho
today=datetime.date.today()
csv_saida=output+'early_natura-mex-padrao_'+str(today.strftime('%d-%m-%Y'))+'.csv'
#Arquivo com números inválidos
errados_saida=dir_num_errados+'numeros-errados_natura-mex-padrao_'+str(today)+'.xlsx'
#Checando arquivos na ingestion
print(os.listdir(ingestion))

['Rep_CarteraApha_22092023_1207.csv']


In [44]:
#Mapeando o arquivo Rep
nome_rep_cartera=''
while nome_rep_cartera=='':
  for arquivo in os.listdir(ingestion):
    if arquivo[:3]=='Rep':
      nome_rep_cartera=arquivo
  time.sleep(5)

In [45]:
concatenado=pd.read_csv(auxiliary+'Personas Concatenado.csv',dtype=object)
concatenado

,CN,'IFE/RFC'
0,4190424,ESMRZY00101115M300
1,4190426,RZRGKR96031409M800
2,4190433,GRMZED93030310M600
3,4190437,MNISSB51070630M000
4,4190438,ANAQCN98121520M600
...,...,...
1925186,5091235,LRSNFL76040620M200
1925187,5091236,NCLPMC84031120M000
1925188,5091237,RBVLES01020322M299
1925189,5091238,CRGRLS04110319H400


In [46]:
df_rep_cartera=pd.read_csv(ingestion+nome_rep_cartera,dtype=object,encoding='utf_8')

##Cópia da carteira principal

In [47]:
rep_cartera=df_rep_cartera.copy()

##Início das Transformações

In [48]:
#Capitalizando coluna de nomes
rep_cartera['Nombre']=rep_cartera['Nombre'].str.title()

In [49]:
#Mesclando Rep_Cartera
rep_cartera_mesclado=rep_cartera.merge(concatenado, left_on='Codigo Persona',right_on='CN',how='left')
rep_cartera_mesclado=rep_cartera_mesclado.drop('CN',axis=1)

In [50]:
#Número de vazios nas duas carteiras
#print('Número de vazios em Cartera: '+str(cartera_mesclado[r"'IFE/RFC'"].isnull().sum()))
print('Número de vazios em RepCartera: '+str(rep_cartera_mesclado[r"'IFE/RFC'"].isnull().sum()))

Número de vazios em RepCartera: 706


In [51]:
#Exportando os Nulos
#rep_cartera_mesclado[rep_cartera_mesclado[r"'IFE/RFC'"].isnull()].to_excel(output + nome_saida_rep_nulos,index=False)

In [52]:
rep_cartera_mesclado.columns

Index(['INDICE', 'Codigo Persona', 'Nombre', 'Numero Pedido',
       'Fecha Vencimiento', 'Dias de credito', 'Dias de morosidad',
       'Temporalidad Agencias', 'Valor Titulo', 'Saldo Principal',
       'Saldo actualizado', 'Estructura Padre', 'Estructura',
       'Direccion Completa', 'Telefono movil', 'Telefono Residencial',
       'Correo electronico personal', 'Referencia Bancaria',
       'Agencia Asignada MC', 'Estatus MC', 'Dias en la instancia actual',
       ''IFE/RFC''],
      dtype='object')

In [53]:
#Formatando colunas numéricas para 2 casas decimais
colunas=['Valor Titulo', 'Saldo Principal','Saldo actualizado']
rep_cartera_mesclado[colunas]=rep_cartera_mesclado[colunas].astype(float)
for coluna in colunas:
  rep_cartera_mesclado[coluna]=rep_cartera_mesclado[coluna].map('{:.2f}'.format)
rep_cartera_mesclado[colunas]

,Valor Titulo,Saldo Principal,Saldo actualizado
0,1368.23,1367.99,1846.79
1,2542.92,2542.92,3178.65
2,2240.88,720.04,972.05
3,854.74,854.30,1067.88
4,1692.89,297.75,401.96
...,...,...,...
7010,1089.45,1089.10,1470.29
7011,1293.41,564.52,762.10
7012,956.40,956.40,1291.14
7013,1635.04,834.39,1126.43


In [54]:
#Apagar IFE nulos
rep_cartera_nao_nulos=rep_cartera_mesclado.dropna(subset=r"'IFE/RFC'")

In [55]:
#Apagar coluna Dias en la instancia actual
rep_cartera_nao_nulos=rep_cartera_nao_nulos.drop('Dias en la instancia actual',axis=1)

###Tratando número de telefones

In [56]:
celular=np.array(rep_cartera_nao_nulos['Telefono movil']).astype(str)
for i in range(len(celular)):
  if len(celular[i])>10:
    celular[i]=str(celular[i])[-10:]
  elif len(celular[i])<10 or celular[i]==''.zfill(10):
    celular[i]=''

#Substituindo números pelos valores da lista
rep_cartera_nao_nulos.loc[:,'Telefono movil']=celular

###Tratamento de Email

In [57]:
  ###Tratamento de Email
  # Aplicando lower e strip na coluna de email
  coluna_email = 'Correo electronico personal'
  rep_cartera_nao_nulos[coluna_email] = rep_cartera_nao_nulos[coluna_email].str.lower().str.strip()
  # Apagando emails não validados
  rep_cartera_nao_nulos[coluna_email].loc[rep_cartera_nao_nulos[coluna_email].str.find('@') < 0] = ''
  rep_cartera_nao_nulos['num @'] = rep_cartera_nao_nulos[coluna_email].str.len() - rep_cartera_nao_nulos[coluna_email].str.replace('@', '').str.len()
  filtro = rep_cartera_nao_nulos['num @'] != 1
  rep_cartera_nao_nulos.loc[filtro, coluna_email] = ''
  rep_cartera_nao_nulos.drop('num @', axis=1, inplace=True)

###Renomear colunas

In [58]:
#Renomeando as colunas
rep_cartera_nao_nulos.columns=['INDICE','CódigoPersona','Nombre','NúmeroPedido','FechaVencimiento','Dias de credito','Dias de morosidad',\
                               'Temporalidad Agencias','ValorTítulo','Saldo Principal','Saldo actualizado','Estructura Padre','Estructura',\
                               'Direccion Completa','Telefono movil','Telefono Residencial','Correo electronico personal','Referencia Bancaria',\
                               'Agencia Asignada MC','Estatus MC','curp']

###Tratar coluna de Nome

In [59]:
coluna_nomes=['Nombre']
for coluna_nome in coluna_nomes:
  rep_cartera_nao_nulos[coluna_nome]=rep_cartera_nao_nulos[coluna_nome].str.strip().str.title()

##Exportando

###Exportando arquivos de números errados

In [60]:
#Criando cópia
num_errados=df_rep_cartera.copy()
#Pegando lista de números
numeros=np.array(num_errados['Telefono movil']).astype(str)
#Criando filtro
length=np.char.str_len(numeros)
filtro1=length<10
filtro2=numeros==''.zfill(10)
filtro=filtro1+filtro2
print(filtro1.sum(),filtro2.sum(),filtro.sum(),len(filtro))
#Aplicando filtro
num_errados=num_errados[filtro]
#Exportando arquivo
num_errados.to_excel(errados_saida,index=False)

237 5 242 7015


###Exportando arquivo tratado

In [61]:
#Exportando arquivo tratado
rep_cartera_nao_nulos.to_csv(csv_saida,index=False)

##Movendo arquivos

In [62]:
#Movendo arquivos para processedfiles
if mover:
  arquivos=[nome_rep_cartera]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento AVON PE PADRAO (sem Collecti)**

## Inicialização

In [63]:
#ATUALIZAR A LISTA DE DATAS QUE INICIAM AS CAMPANHAS
mover=True
tem_descuentos=False

In [64]:
import pandas as pd
import numpy as np
import os
import datetime
import time

###Verificar se tem Descuentos

In [65]:
if datetime.date.today()<=datetime.date(2023,9,30):
  tem_descuentos=True

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
carteira='Avon PE'

today=datetime.date.today()
#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(today.year),str(today.month).zfill(2),today.strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
ingestion=diretorio+'ingestion/'
processedfiles=diretorio+'processedfiles/'
dir_descuentos=diretorio+'Descuentos/'
auxiliary=diretorio+'auxiliary/'

#Nome do arquivo de saída com caminho
csv_saida=output+'early_avon-pe-padrao_'+str(today.strftime('%d-%m-%Y'))+'.csv'

##Identificar arquivos da auxiliary

In [68]:
#Emails e Calendario
for arquivo in os.listdir(auxiliary):
    if arquivo.find('Emails')>=0:
        arquivo_emails=auxiliary+arquivo
        nome_emails=arquivo
    elif arquivo.find('Calendario')>=0:
        arquivo_calendario=auxiliary+arquivo
        nome_calendario=arquivo

##Importar arquivo Descuentos

In [69]:
if tem_descuentos:
  #Identificando o arquivo mais recente da pasta Descuentos
  df_arquivo_recente=pd.DataFrame({'Nome':os.listdir(dir_descuentos)})
  tempo_modif=np.empty(len(df_arquivo_recente))
  i=0
  for arquivo in os.listdir(dir_descuentos):
    tempo_modif[i]=os.path.getmtime(dir_descuentos+arquivo)
    i+=1
  df_arquivo_recente['Tempo de Modificacao']=tempo_modif
  df_arquivo_recente=df_arquivo_recente.sort_values(by='Tempo de Modificacao',ascending=False)
  df_arquivo_recente.index=range(len(df_arquivo_recente))
  print(df_arquivo_recente)
  nome_arquivo_descuentos=str(df_arquivo_recente['Nome'][0])
  arquivo_descuentos=dir_descuentos+nome_arquivo_descuentos
  print(arquivo_descuentos)

                                                Nome  Tempo de Modificacao
0  DESCUENTOS COLLECTION HISOFI 15 AL 30 DE SET 2...          1.695071e+09
/content/drive/MyDrive/Processamentos/Tratamento/Avon PE/Descuentos/DESCUENTOS COLLECTION HISOFI 15 AL 30 DE SET 2023.xlsx


##Importar arquivo de Calendário

In [70]:
df_calendario=pd.read_excel(arquivo_calendario,dtype=object)
#Definir último dia
if today.weekday()==0:
  dias=3
else:
  dias=1
ultimo=today-datetime.timedelta(days=dias)

##Definir se Tem Collecti

In [71]:
#Checando se tem collecti
if str(ultimo) in np.array(df_calendario['Data'].astype(str)):
  tem_collecti=True
  zona=np.array(df_calendario.loc[df_calendario['Data']==str(ultimo),'Zona']).astype(int)
else:
  tem_collecti=False
print(tem_collecti)
tem_collecti=False

False


##Ingestion

In [72]:
#Esperando ler 1 ou 2 arquivos na ingestion (depende do parâmetro 'tem_especial')
if tem_collecti:
  num_arquivo=2
else:
  num_arquivo=1

while len(os.listdir(ingestion))!=num_arquivo:
  time.sleep(5)
print(os.listdir(ingestion))

['SALDOS_C.TXT']


##Identificar arquivos da ingestion

In [73]:
#Identificando arquivo COLLECTI e SALDOS
for arquivo in os.listdir(ingestion):
    if arquivo.find('COLLECTI')>=0:
        arquivo_txt=ingestion+arquivo
        nome_txt=arquivo
    elif arquivo.find('SALDOS_C')>=0:
        arquivo_saldos=ingestion+arquivo
        nome_saldos=arquivo

##Importando Arquivos

In [74]:
#Carregando Planilha Collecti
if tem_collecti:
  df_txt=pd.read_csv(arquivo_txt,dtype=object,sep='\t', lineterminator='\r',encoding='unicode_escape',encoding_errors='ignore')
#Carregando Planilha Saldos
df_saldos=pd.read_csv(arquivo_saldos,dtype=object,sep='\t', lineterminator='\r',encoding='unicode_escape',encoding_errors='ignore')
#Carregando planilha de descontos se tiver
if tem_descuentos:
  df_descuentos=pd.read_excel(arquivo_descuentos,dtype=object)
#Carregando planilha de Emails
df_emails=pd.read_csv(arquivo_emails,dtype=object)

##Início das Tranformações

###Juntar tabela de Saldos e Collecti

####Criando cópias

In [75]:
#Criando cópia do txt para tratar
if tem_collecti:
  txt=df_txt.copy()
saldos=df_saldos.copy()
emails=df_emails.copy()
if tem_descuentos:
  descuentos=df_descuentos.copy()

###Atualizar e Exportar planilha de Emails com o arquivo COLLECTI novo

In [76]:
#Tratando coluna ZONA
if tem_collecti:
  txt['ZONA']=txt['ZONA'].str.replace('\n','').str.strip()
  emails['ZONA']=emails['ZONA'].str.replace('\n','').str.strip()
  #Validação das ZONAs
  txt_zonas=txt.loc[txt['ZONA']!='','ZONA'].unique().astype(int)  #zona do calendário está em int tbm
  for txt_zona in txt_zonas:
    if txt_zona not in zona:
      print('ZONA do Calendário não está no COLLECTI')
      import sys
      sys.exit()
  for x in zona:
    if not x in txt_zonas:
      print('ZONA do COLLECTI não está no Calendário')
      sys.exit()

In [77]:
if tem_collecti:
  #Retirar Emails do Collecti novo do arquivo antigo de Emails
  filtro=np.logical_not(np.isin(emails.ZONA.astype(int),zona))  #transformar em int tira os epaços antes
  emails=emails[filtro]
  #Agrupar emails por external ID
  novo=pd.concat([emails,txt])
  print('Antes',len(novo))
  novo=novo.groupby(['ZONA','CARTA','SEC','EQUIPO','TERRITORIO'],as_index=False).last()
  print('Depois',len(novo))

###Selecionar colunas da Tabela de Emails

In [78]:
emails=emails[['ZONA','CARTA','SEC','EQUIPO','TERRITORIO','E-MAIL']]

####Tratando coluna de Zonas

In [79]:
#Tratando ZONA
saldos['ZONA']=saldos['ZONA'].str.replace('\n','').str.strip()

####Eliminando ZONAS da tabela SALDOS que estão na COLLECTI

In [80]:
if tem_collecti:
  filtro=np.logical_not(np.isin(saldos.ZONA,np.unique(txt.ZONA)))
  saldos=saldos[filtro]
  print(len(saldos)-filtro.sum(),'linhas excluídas de',len(saldos))

In [81]:
if tem_collecti:
  saldos=pd.concat([txt,saldos],join='inner')

###Tratando tabela de Descuentos se houver e Substituindo Deudas com Desconto

In [82]:
if tem_descuentos:
  #Deletando colunas indesejadas, agrupando por TERRITORIO e pegando o valor com menor desconto se houver duplicado
  coluna_importe='Monto a cancelar hasta el 30/09'
  colunas=['TERRITORIO',coluna_importe]
  descuentos=descuentos.sort_values('% DESCUENTO').groupby('TERRITORIO',as_index=False).first()[colunas]
  descuentos['TERRITORIO']=descuentos['TERRITORIO'].astype(str)
  #Merge com tabela principal
  saldos=saldos.merge(descuentos,on='TERRITORIO',how='left')
  #Renomeando coluna Importe por DEUDA A PAGAR
  saldos=saldos.rename(columns={coluna_importe:'DEUDA A PAGAR'})
  #Preenchendo DEUDA A PAGAR vazios POR DEU+PER
  filtro=saldos['DEUDA A PAGAR'].isna()
  saldos.loc[filtro,'DEUDA A PAGAR']=saldos.loc[filtro,'DEU+PER']
else:
  saldos['DEUDA A PAGAR']=saldos['DEU+PER']

###Tratamento da Carteira

####Apagar linhas nulas

In [83]:
#Apagando linhas nulas
saldos.dropna(subset='TERRITORIO',inplace=True)

####Aplicando strip em todas as colunas

In [84]:
#Aplicando strip em todas as colunas e substituindo } por '' e substituindo nan por ''
for coluna in saldos.columns.to_list():
  saldos[coluna]=saldos[coluna].astype(str)
  saldos[coluna]=saldos[coluna].str.strip().str.replace('}','')
  saldos[coluna]=saldos[coluna].str.replace('nan','')

<ipython-input-84-92f10093bc77>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  saldos[coluna]=saldos[coluna].str.strip().str.replace('}','')


####Formatando as colunas numéricas apra 2 casas

In [85]:
#Formatando colunas numéricas para 2 casas decimais
colunas=['DEUDA INICIAL','DEUDA','DEU+PER','DEUDA A PAGAR']
for coluna in colunas:
  #Transformar vazios em 0
  saldos[coluna]=saldos[coluna].apply(lambda x: '0' if str(x)=='' else x)
  #Se último caractere for letra, retirar ele
  saldos[coluna]=saldos[coluna].apply(lambda x: str(x)[:-1] if str(x)[-1].isalpha() else str(x))
  #Formatando em número de 2 casas decimais
  saldos[coluna]=pd.to_numeric(saldos[coluna])
  saldos[coluna].fillna(0,inplace=True)
  saldos[coluna]=saldos[coluna].map('{:.2f}'.format)
  saldos[coluna]=saldos[coluna].astype(object)

###Tratamento de Número de Celular

In [86]:
import re
colunas=['TELEFONO 1', 'TELEFONO 2', 'TELEFONO 3', 'TELEFONO 4']
#Tirar espaços
for coluna in colunas:
  #Tirar caracteres não numéricos
  saldos[coluna]=saldos[coluna].apply(lambda x: re.sub('[^0-9]', '', str(x)))
  #Tirar números com mais de 12 dígitos ou menos de 8 dígitos
  saldos[coluna]=saldos[coluna].apply(lambda x: '' if len(x)>12 or len(x)<8 else x)
  #Pegar últimos 9 dígitos de números com mais de 9 dígitos e completar com dígito 9
  saldos[coluna]=saldos[coluna].apply(lambda x: x[-9:] if len(x)>9 else ('9'+ x if len(x)==8 else x))
  #Apagar números que não começam com 9
  filtro=saldos[coluna]!=''
  saldos.loc[filtro,coluna]=saldos.loc[filtro,coluna].apply(lambda x: '' if x[0]!='9' else x)
#Completar TELEFONO 1 nulos pelo 2, 3 e 4
filtro=saldos['TELEFONO 1']==''
saldos.loc[filtro,'TELEFONO 1']=saldos.loc[filtro,'TELEFONO 2']
filtro=saldos['TELEFONO 1']==''
saldos.loc[filtro,'TELEFONO 1']=saldos.loc[filtro,'TELEFONO 3']
filtro=saldos['TELEFONO 1']==''
saldos.loc[filtro,'TELEFONO 1']=saldos.loc[filtro,'TELEFONO 4']

###Tratando as colunas ZONA, CARTA, SEC, EQUIPO, TERRITORIO de emails e saldos

In [87]:
colunas=['ZONA', 'CARTA', 'SEC', 'EQUIPO', 'TERRITORIO']
for coluna in colunas:
    saldos[coluna]=saldos[coluna].astype(str).str.strip()
    emails[coluna]=emails[coluna].astype(str).str.strip()
saldos['EXTERNAL']=saldos['ZONA']+' '+saldos['CARTA']+' '+saldos['SEC']+' '+saldos['EQUIPO']+' '+saldos['TERRITORIO']
saldos['EXTERNAL']=saldos['EXTERNAL'].astype(str).str.strip()
emails['EXTERNAL']=emails['ZONA']+' '+emails['CARTA']+' '+emails['SEC']+' '+emails['EQUIPO']+' '+emails['TERRITORIO']
emails['EXTERNAL']=emails['EXTERNAL'].astype(str).str.strip()
emails.drop(colunas,axis=1,inplace=True)

###Mesclando coluna de Email

In [88]:
print('Antes',len(saldos))
saldos=saldos.merge(emails,on='EXTERNAL',how='left')
print('Depois',len(saldos))

Antes 66277
Depois 66277


###Tratamento de Email

In [89]:
#Aplicando lower na coluna de email
coluna_email='E-MAIL'
saldos[coluna_email]=saldos[coluna_email].str.lower().str.strip()
#Apagando emails não validados
saldos[coluna_email].loc[saldos[coluna_email].str.find('@')<0]=''
saldos['num @']=saldos[coluna_email].str.len()-saldos[coluna_email].str.replace('@','').str.len()
filtro=saldos['num @']!=1
saldos.loc[filtro,coluna_email]=''
saldos.drop('num @',axis=1,inplace=True)

###Preenchendo ULT.A#O FACTURACION Vazio


In [90]:
saldos.loc[saldos['ULT.A#O FACTURACION']=='','ULT.A#O FACTURACION']='2018'

###Tratar coluna de Nome

In [91]:
coluna_nomes=['APELLIDOS','NOMBRES']
for coluna_nome in coluna_nomes:
  saldos[coluna_nome]=saldos[coluna_nome].str.strip().str.title()

##Exportando

In [92]:
#Carteira Tratada
saldos.to_csv(csv_saida,index=False)
if tem_collecti:
  #Arquivo de Emails.csv
  novo.to_csv(arquivo_emails,index=False)

##Movendo Arquivos da Ingestion

In [93]:
if mover:
  if tem_collecti:
    arquivos=[nome_txt,nome_saldos]
  else:
     arquivos=[nome_saldos]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento PRONTO**

## Inicialização

In [94]:
mover=True

In [95]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
carteira='Pronto'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
ingestion=diretorio+'ingestion/'
processedfiles=diretorio+'processedfiles/'

#Nome do arquivo de saída com caminho
csv_saida=output+'early_pronto-carteira-1_'+str(datetime.date.today().strftime('%d-%m-%Y'))+'.csv'

In [98]:
#Esperando ler pelo menos 1 arquivo na ingestion
while len(os.listdir(ingestion))==0:
  time.sleep(5)
print(os.listdir(ingestion))

['Asignacion20230922.xlsx']


In [99]:
#Identificando a planilha Asignacion
for arquivo in os.listdir(ingestion):
    if arquivo.find('Asignacion')>=0:
        nome_asignacion=arquivo

arquivo_asignacion=ingestion + nome_asignacion

In [100]:
#Carregando Asignacion
df_asignacion=pd.read_excel(arquivo_asignacion,dtype=object)

## Início das Transformações

### Apagar coluna de índice

In [101]:
#Criando cópia do arquivo principal
asignacion=df_asignacion.copy()
print(asignacion.columns)
print(asignacion.info())

#Deletando coluna de índices
del asignacion[asignacion.columns[0]]
len(asignacion.columns)

Index(['Unnamed: 0', 'Cedula', 'Cliente', 'cartera', 'Apellido1', 'Apellido2',
       'Nombre1', 'Nombre2', 'Edad', 'Sexo', 'CiuNom', 'DepNom', 'CPostal',
       'Producto', 'Numero', 'vencimiento', 'DeudaCliente', 'DeudaMin',
       'celular', 'email'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4480 entries, 0 to 4479
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    4480 non-null   object
 1   Cedula        4480 non-null   object
 2   Cliente       4480 non-null   object
 3   cartera       4480 non-null   object
 4   Apellido1     4479 non-null   object
 5   Apellido2     4301 non-null   object
 6   Nombre1       4480 non-null   object
 7   Nombre2       3934 non-null   object
 8   Edad          4480 non-null   object
 9   Sexo          4480 non-null   object
 10  CiuNom        4428 non-null   object
 11  DepNom        4435 non-null   object
 12  CPostal       3733 non-

19

In [102]:
#Converter data pro formato brasileiro (não precisa)
'''asignacion['vencimiento']=pd.to_datetime(asignacion['vencimiento'])
asignacion['vencimiento']=asignacion['vencimiento'].dt.strftime('%d/%m/%Y')
asignacion['vencimiento']'''

"asignacion['vencimiento']=pd.to_datetime(asignacion['vencimiento'])\nasignacion['vencimiento']=asignacion['vencimiento'].dt.strftime('%d/%m/%Y')\nasignacion['vencimiento']"

### Adicionando as colunas: 'Moneda', 'Cuot minima deuda cliente' e 'Cantidad maxima cuotas' e reordenando as colunas

In [103]:
#Adicionando colunas 'Moneda', 'Cuot minima deuda cliente' e 'Cantidad maxima cuotas' e reordenando as colunas
asignacion['Moneda']='UYU'
asignacion['Cuota minima deuda cliente']=200
asignacion['Cantidad maxima cuotas']=1
asignacion=pd.concat([asignacion.loc[:,:'CPostal'],asignacion.loc[:,'Moneda':'Cantidad maxima cuotas'],asignacion.loc[:,'Producto':'email']],join='outer',axis=1)
asignacion.columns

Index(['Cedula', 'Cliente', 'cartera', 'Apellido1', 'Apellido2', 'Nombre1',
       'Nombre2', 'Edad', 'Sexo', 'CiuNom', 'DepNom', 'CPostal', 'Moneda',
       'Cuota minima deuda cliente', 'Cantidad maxima cuotas', 'Producto',
       'Numero', 'vencimiento', 'DeudaCliente', 'DeudaMin', 'celular',
       'email'],
      dtype='object')

### Formatando 'DeudaCliente','DeudaMin' para 2 casas decimais

In [104]:
#Formatando números para 2 casas decimais
colunas=['DeudaCliente','DeudaMin']
asignacion[colunas]=asignacion[colunas].astype(float)
asignacion[colunas]=asignacion[colunas].round(2)
for coluna in colunas:
  asignacion[coluna]=asignacion[coluna].map('{:.2f}'.format)
asignacion[colunas]

,DeudaCliente,DeudaMin
0,13173.37,3724.23
1,4547.43,55.30
2,6339.79,6339.79
3,2680.51,2680.51
4,16314.47,8363.31
...,...,...
4475,10926.30,3873.59
4476,8003.26,4052.40
4477,4956.68,2478.34
4478,8393.06,4271.29


### Retirando - e . da coluna Cedula

In [105]:
#Retirando - e . da coluna Cedula
asignacion['Cedula']=asignacion['Cedula'].str.replace('.','').str.replace('-','')
asignacion['Cedula']

<ipython-input-105-ef483c74cf5f>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  asignacion['Cedula']=asignacion['Cedula'].str.replace('.','').str.replace('-','')


0       38942840
1       32586032
2       42329723
3       32132726
4       25019925
          ...   
4475    50442650
4476    53854135
4477    54476251
4478    44896075
4479    53998000
Name: Cedula, Length: 4480, dtype: object

###Renomeando as colunas

In [106]:
asignacion.columns=['Cedula','Cliente','cartera','Apellido1','Apellido2','Nombre1','Nombre2',\
                    'Edad','Sexo','CiuNom','DepNom','CPostal','Moneda','Cuota minima deuda cliente',\
                    'Cantidad maxima cuotas','Producto','numero','Vencimiento','Deudacliente','Deudamin','Celular','email']

###Tratamento de Email

In [107]:
#Aplicando lower e strip na coluna de email
coluna_email='email'
asignacion[coluna_email]=asignacion[coluna_email].str.lower().str.strip()
#Apagando emails não validados
asignacion.loc[asignacion[coluna_email].str.find('@')<0,coluna_email]=''
asignacion['num @']=asignacion[coluna_email].str.len()-asignacion[coluna_email].str.replace('@','').str.len()
filtro=asignacion['num @']!=1
asignacion.loc[filtro,coluna_email]=''
asignacion.drop('num @',axis=1,inplace=True)

###Tratamento do Celular (tirar 0 à esquerda)


In [108]:
asignacion['Celular']=asignacion['Celular'].astype(int)

###Tratar coluna de Nome

In [109]:
coluna_nomes=['Apellido1','Apellido2','Nombre1','Nombre2']
for coluna_nome in coluna_nomes:
  asignacion[coluna_nome]=asignacion[coluna_nome].str.strip().str.title()

## Exportando

In [110]:
asignacion.to_csv(csv_saida,index=False)

## Movendo arquivos processados

In [111]:
if mover:
  arquivos=[nome_asignacion]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento PORTO ESSENCIAL**

In [112]:
import pandas as pd
import numpy as np
import datetime
import os
import time

##Mover arquivos ao finalizar o tratamento?

In [113]:
mover=True

In [114]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
carteira='Porto Essencial'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
processedfiles=diretorio+'processedfiles/'
ingestion=diretorio+'ingestion/'

#Nome do arquivo de saída com caminho
csv_saida=output+'early_porto-essencial_'+str(datetime.date.today().strftime('%d-%m-%Y'))+'.csv'

In [116]:
#Esperando ler 2 arquivos na ingestion
while len(os.listdir(ingestion))!=2:
  time.sleep(5)
print(os.listdir(ingestion))

['TABELA_ESSENCIAL_HISOFI.xlsx', 'ESC_ESSEN_565.CSV']


In [117]:
#Lendo os 2 arquivos
for arquivo in os.listdir(ingestion):
  if arquivo.find('ESC_ESSEN')>=0:
    print('ESC_ESSEN:',arquivo)
    nome_essen=arquivo
    esc_essen=pd.read_csv(ingestion+arquivo,dtype=object)
  elif arquivo.find('TABELA_ESSENCIAL')>=0:
    nome_tab_essen=arquivo
    tab_essen=pd.read_excel(ingestion+arquivo,dtype=object)
    print('TABELA_ESSENCIAL:',arquivo)

TABELA_ESSENCIAL: TABELA_ESSENCIAL_HISOFI.xlsx
ESC_ESSEN: ESC_ESSEN_565.CSV


##Início dos Tratamentos

### Tratamento de TABELA_ESSENCIAL

In [118]:
#Listando colunas repetidas da tabela auxiliar diferente de NR_RAS
col_repetida=[]
for coluna in tab_essen.columns.to_list():
    if coluna in esc_essen.columns.to_list() and coluna!='NR_RAS':
        col_repetida.append(coluna)

print('Colunas repetidas:') #Resultados anteriores: ['DT_FECH_SINISTRO', 'VR_DESPESAS_SIN']
print(col_repetida)

Colunas repetidas:
['DT_FECH_SINISTRO', 'VR_DESPESAS_SIN']


In [119]:
#Capitalizando nomes
tab_essen['NM_GARANTIDO']=tab_essen['NM_GARANTIDO'].str.title()

In [120]:
#Excluindo colunas repetidas de tab_essen
tab_essen.drop(col_repetida,axis=1,inplace=True)

In [121]:
#Tratando '..' na coluna NR_CELULAR_PORTO de tab_essen
tab_essen['NR_CELULAR_PORTO']=tab_essen['NR_CELULAR_PORTO'].apply(lambda x: '' if str(x)=='..' else x)

In [122]:
#Excluindo colunas indesejadas
col_final=['NR_RAS','NR_COD_ESCRITORIO_NOVO','DT_FECH_SINISTRO','QT_DIAS_ATRASO','VR_DESPESAS_SIN','VR_EM_ATRASO_COM_JUROS','PC_DES_AVISTA',\
                    'PC_DES_2_3_VEZES','PC_DES_4_6_VEZES','PC_DES_7_12_VEZES','PC_DES_13_20_VEZES','PC_DESC_20_VEZES','VR_VALOR_COM_DESPESA','PROPENSAO',\
                    'NM_GARANTIDO','NR_CPF_V2','NR_CNPJ_V2','NR_CELULAR_PORTO','CD_UF','VR_COBERTURA_ALUGUEL','EMAIL_1','DT_NASCIMENTO_INQUILINO']
excluir=[]
for coluna in tab_essen.columns.to_list():
  if coluna not in col_final:
    excluir.append(coluna)
tab_essen=tab_essen.drop(excluir, axis=1)

In [123]:
#Ordenando por email e agrupando por NR_RAS
tab_essen=tab_essen.sort_values('EMAIL_1',ascending=False)
tab_essen=tab_essen.groupby('NR_RAS',as_index=False).first()

In [124]:
#Convertendo datas pro formato brasileiro
for coluna in tab_essen.select_dtypes(include='datetime').columns.tolist():
    tab_essen[coluna]=tab_essen[coluna].apply(lambda x: str(x)[:10])
    tab_essen[coluna]=pd.to_datetime(tab_essen[coluna])
    tab_essen[coluna]=tab_essen[coluna].dt.strftime('%d/%m/%Y')

### Tratamento de ESC_ESSEN

In [125]:
#Tratando com separador de milhares como '.' e de decimal como ','
tratar_ponto_virgula=['VR_EM_ATRASO_COM_JUROS','VR_VALOR_COM_DESPESA']

for coluna in tratar_ponto_virgula:
  esc_essen[coluna]=esc_essen[coluna].apply(lambda x: str(x).replace('.',''))
  esc_essen[coluna]=esc_essen[coluna].apply(lambda x: str(x).replace(',','.'))
  esc_essen[coluna]=esc_essen[coluna]=esc_essen[coluna]=esc_essen[coluna].astype(float)
  print(esc_essen[coluna])

0        11129.81
1         1180.43
2         4518.20
3         3005.58
4         1752.19
           ...   
12332     3013.37
12333     3748.33
12334    10085.20
12335      977.96
12336     2644.92
Name: VR_EM_ATRASO_COM_JUROS, Length: 12337, dtype: float64
0        10295.77
1          944.71
2         3569.19
3         2405.40
4         1402.30
           ...   
12332     3009.17
12333     3741.36
12334    10064.12
12335      976.14
12336     2640.00
Name: VR_VALOR_COM_DESPESA, Length: 12337, dtype: float64


### Mesclando as duas tabelas

In [126]:
#Fazendo o merge das Tabelas
merged=esc_essen.merge(tab_essen,on='NR_RAS',how='inner')

In [127]:
#Eliminando colunas indesejadas
merged=merged[col_final]

###Tratando os números de telefone e email

In [128]:
#Validação de Celular
col_celular='NR_CELULAR_PORTO'
filtro=np.char.str_len(np.array(merged[col_celular]).astype(str))!=11
merged.loc[filtro,col_celular]=''
#Aplicando lower e strip na coluna de email
coluna_email='EMAIL_1'
merged[coluna_email]=merged[coluna_email].str.lower().str.strip()
#Apagando emails não validados
merged[coluna_email].loc[merged[coluna_email].str.find('@')<0]=''
merged['num @']=merged[coluna_email].str.len()-merged[coluna_email].str.replace('@','').str.len()
filtro=merged['num @']!=1
merged.loc[filtro,coluna_email]=''
merged.drop('num @',axis=1,inplace=True)

<ipython-input-128-371e18857a3c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged[coluna_email].loc[merged[coluna_email].str.find('@')<0]=''


###Tratar coluna de Nome

In [129]:
coluna_nomes=['NM_GARANTIDO']
for coluna_nome in coluna_nomes:
  merged[coluna_nome]=merged[coluna_nome].str.strip().str.title()

##Exportando

In [130]:
#Exportando o arquivo para Output
merged.to_csv(csv_saida,index=False)

##Movendo arquivos

In [131]:
#Movendo arquivos para processedfiles
if mover:
  arquivos=[nome_tab_essen,nome_essen]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento RISEL**

## Inicialização

In [132]:
mover=True

In [133]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [134]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [135]:
carteira='Risel'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
ingestion=diretorio+'ingestion/'
processedfiles=diretorio+'processedfiles/'
os.makedirs(output,exist_ok=True)
os.makedirs(ingestion,exist_ok=True)
os.makedirs(processedfiles,exist_ok=True)

today=datetime.date.today()
#Nome do arquivo de saída com caminho
csv_saida=output+'early_risel_'+str(today.strftime('%d-%m-%Y'))+'.csv'

In [136]:
#Esperando ler 1
num_arquivo=1

while len(os.listdir(ingestion))!=num_arquivo:
  time.sleep(5)
print(os.listdir(ingestion))

['1691077668699template_sofi.xlsx']


In [137]:
#Identificando as planilhas EXCLUSIVA e ESPECIAL
for arquivo in os.listdir(ingestion):
    if arquivo.find('template_sofi')>=0:
      nome_exclusiva=arquivo
      arquivo_exclusiva=ingestion+arquivo

In [138]:
#Carregando Planilhas
df_endereco=pd.read_excel(arquivo_exclusiva,dtype=object,sheet_name='ENDERECO')
df_titulos=pd.read_excel(arquivo_exclusiva,dtype=object,sheet_name='TITULOS')

In [139]:
#Criar cópias
endereco=df_endereco.copy()
titulos=df_titulos.copy()

## Início das Transformações

###Tratar Tabela de ENDEREÇO

####Excluir colunas repetidas da tabela ENDEREÇO

In [140]:
excluir=[]
for coluna in endereco.columns.to_list():
  if coluna in titulos.columns.to_list():
    excluir.append(coluna)
endereco.drop(excluir, axis=1, inplace=True)

####Agrupar por COD

In [141]:
endereco=endereco.groupby('COD',as_index=False).last()

###Mesclar tabelas

In [142]:
merged=titulos.merge(endereco,left_on='CODCLIENTE',right_on='COD',how='left')
merged.drop('COD', axis=1, inplace=True)

###Tratamento dos números de telefone

In [143]:
merged['TELEFONE']=merged['TELEFONE'].str.replace(' ','').str.replace('-','').str.replace(')','').str.replace('(','')

<ipython-input-143-aff8f3d8da88>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged['TELEFONE']=merged['TELEFONE'].str.replace(' ','').str.replace('-','').str.replace(')','').str.replace('(','')


###Tratamento de Endereço

In [144]:
#Preenchendo vazios
colunas=['ENDERECO','BAIRRO','NUMEND','COMPLMENTO']
for coluna in colunas:
  merged[coluna]=merged[coluna].fillna('')
#Concatenando endereço
merged['ENDERECO COMPLETO']=merged['ENDERECO'] + ' ' + merged['BAIRRO'] + ' ' + merged['NUMEND'] + ' ' + merged['COMPLMENTO']
merged.reset_index(drop=True,inplace=True)
#Remover Espaços duplos ou superiores
for i in range(len(merged)):
  split=merged.loc[i,'ENDERECO COMPLETO'].split(' ')  #aplicar split por ' '
  split=[s for s in split if s!='']  #remover strings vazias do array
  merged.loc[i,'ENDERECO COMPLETO']=' '.join(split) #juntar strings de split sem os ''

###Adicionar coluna de Payload

In [145]:
merged['payload']=merged['FILIAL'].astype(str) + ';' + merged['ESTAB'].astype(str)

###Tratamento de Email

In [146]:
#Escolher o primeiro email sem @risel
#Split em email
merged['EMAIL']=merged['EMAIL'].str.split(';')
#Flatten em email
merged=merged.explode('EMAIL')
#Apagar emails com @risel
filtro=merged['EMAIL'].str.find('@risel')>=0
merged.loc[filtro,'EMAIL']=''
#Ordenar por email desc
merged=merged.sort_values('EMAIL',ascending=False)
#Manter o primeiro email
merged=merged.drop_duplicates('DOC',keep='first')

In [147]:
# Aplicando lower e strip na coluna de email
coluna_email = 'EMAIL'
merged[coluna_email] = merged[coluna_email].str.lower().str.strip()
# Apagando emails não validados
merged[coluna_email].loc[merged[coluna_email].str.find('@') < 0] = ''
merged['num @'] = merged[coluna_email].str.len() - merged[coluna_email].str.replace('@', '').str.len()
filtro = merged['num @'] != 1
merged.loc[filtro, coluna_email] = ''
merged.drop('num @', axis=1, inplace=True)

###Tratar coluna de Nome

In [148]:
coluna_nomes=['CLIENTE']
for coluna_nome in coluna_nomes:
  merged[coluna_nome]=merged[coluna_nome].str.strip().str.title()

###Renomear Colunas

In [149]:
merged=merged.rename(columns={'DOC':'external_id', 'CNPJ':'document', 'CLIENTE':'full_name', 'TELEFONE':'phone', \
                       'EMAIL':'email', 'VALOR':'amount', 'CODCLIENTE':'product',\
                       'VENCIMENTO':'due_date', 'ENDERECO COMPLETO':'address', 'CEP':'zipcode', 'CIDADE':'city', 'UF':'state',\
                       'EMISSAO':'since', 'payload':'payload'})

## Exportando

In [150]:
merged.to_csv(csv_saida,index=False)

## Movendo Somente as Carteiras Exclusivas

In [151]:
if mover:
  arquivos=[nome_exclusiva]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento PORTO BATIMENTO (sem ser Segunda)**

In [152]:
import pandas as pd
import numpy as np
import datetime
import os
import time
import sys

In [153]:
today=datetime.date.today()
#Checando se não é segunda
if today.weekday()==0:
  print('Hoje é segunda! Rodar outro script!')
  sys.exit()

##Mover arquivos ao finalizar o tratamento?

In [154]:
mover=True

In [155]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [156]:
carteira='Porto Batimento'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
processedfiles=diretorio+'processedfiles/'
ingestion=diretorio+'ingestion/'
auxiliary=diretorio+'auxiliary/'
#Nome do arquivo de saída com caminho
csv_saida=output+'early_porto-batimento_'+str(today.strftime('%d-%m-%Y'))+'.csv'

In [157]:
#Esperando ler 2 arquivos na ingestion
while len(os.listdir(ingestion))!=2:
  time.sleep(5)
print(os.listdir(ingestion))

['TB_DIST_HISOFI.xlsx', 'ESC_565.CSV']


In [158]:
#Lendo os arquivos da ingestion (TB_DIST e ESC)
for arquivo in os.listdir(ingestion):
  if arquivo.find('TB')>=0:
    nome_tb=arquivo
    df_tb=pd.read_excel(ingestion+arquivo,dtype=object)
    print(arquivo,'importado com sucesso!')
  elif arquivo.find('ESC')>=0:
    nome_esc=arquivo
    df_esc=pd.read_csv(ingestion+arquivo,dtype=object)
    print(arquivo,'importado com sucesso!')
#Lendo tabela auxiliar concatenada
df_concatenado=pd.read_csv(auxiliary+os.listdir(auxiliary)[0],dtype=object)
print(os.listdir(auxiliary)[0],'importado com sucesso!')

TB_DIST_HISOFI.xlsx importado com sucesso!
ESC_565.CSV importado com sucesso!
Concatenado.csv importado com sucesso!


In [159]:
#Criando cópia dos arquivos importados para fazer alteração
tabela=df_tb.copy()
concatenado=df_concatenado.copy()
esc=df_esc.copy()

In [160]:
concatenado=pd.concat([concatenado,tabela],join='inner').reset_index().drop('index',axis=1)

In [161]:
col_final=['NR_RAS','NR_COD_ESCRITORIO_NOVO','DT_FECH_SINISTRO','QT_DIAS_ATRASO','VR_DESPESAS_SIN','VR_EM_ATRASO_COM_JUROS','PC_DES_AVISTA','PC_DES_2_3_VEZES','PC_DES_4_6_VEZES',\
           'PC_DES_7_12_VEZES','PC_DES_13_20_VEZES','PC_DESC_20_VEZES','VR_VALOR_COM_DESPESA','PROPENSAO','NM_GARANTIDO','NR_CPF_V2','NR_CNPJ_V2','NR_CELULAR_PORTO','CD_UF','VR_COBERTURA_ALUGUEL',\
           'EMAIL_1','DT_NASCIMENTO_INQUILINO']

##Início dos Tratamentos

### Tratamento das Tabelas Auxiliares

In [162]:
#Ordenando por Email e agrupando
concatenado=concatenado.sort_values('EMAIL_1',ascending=False)
concatenado=concatenado.groupby('NR_RAS',as_index=False).first()

In [163]:
#Capitalizando nomes
concatenado['NM_GARANTIDO']=concatenado['NM_GARANTIDO'].str.title()

In [164]:
#Tratando '..' na coluna NR_CELULAR_PORTO de concatenado
concatenado['NR_CELULAR_PORTO']=concatenado['NR_CELULAR_PORTO'].apply(lambda x: '' if str(x)=='..' else x)

In [165]:
#Deixando 2 dígitos decimais
concatenado['VR_COBERTURA_ALUGUEL']=concatenado['VR_COBERTURA_ALUGUEL'].astype(float)
concatenado['VR_COBERTURA_ALUGUEL']=concatenado['VR_COBERTURA_ALUGUEL'].round(2)
concatenado['VR_COBERTURA_ALUGUEL']=concatenado['VR_COBERTURA_ALUGUEL'].map('{:.2f}'.format)

In [166]:
#Tratando números de telefone
concatenado['NR_CELULAR_PORTO']=concatenado['NR_CELULAR_PORTO'].apply(lambda x: str(x).split('.')[0] if str(x).find('.')>=0 else x)

### Tratamento de ESC

In [167]:
#Excluindo colunas indesejadas
excluir=[]
for coluna in esc.columns.to_list():
  if coluna not in col_final:
    excluir.append(coluna)
esc=esc.drop(excluir, axis=1)
esc.head()

,NR_RAS,NR_COD_ESCRITORIO_NOVO,DT_FECH_SINISTRO,QT_DIAS_ATRASO,VR_DESPESAS_SIN,VR_EM_ATRASO_COM_JUROS,PC_DES_AVISTA,PC_DES_2_3_VEZES,PC_DES_4_6_VEZES,PC_DES_7_12_VEZES,PC_DES_13_20_VEZES,PC_DESC_20_VEZES,VR_VALOR_COM_DESPESA,PROPENSAO
0,0746 2003 002317,565,29/03/2004,7121,"0,00","14.903,00",99.33%,99.33%,99.10%,94.38%,89.89%,85.61%,"2.845,63",1-Altamente propenso
1,0746 2003 002515,565,22/09/2003,7310,"0,00","4.500,86",97.78%,97.78%,97.78%,97.78%,94.91%,90.39%,"822,46",1-Altamente propenso
2,0746 2003 002520,565,10/01/2005,6834,"1.068,05","44.948,26",99.78%,99.78%,99.78%,99.78%,99.78%,99.78%,"10.242,91",1-Altamente propenso
3,0746 2003 002530,565,03/09/2003,7329,"0,00","8.351,37",98.80%,98.80%,98.80%,98.67%,93.97%,89.50%,"1.519,35",1-Altamente propenso
4,0746 2003 002547,565,16/10/2003,7286,"0,00","17.236,08",99.42%,99.42%,99.42%,95.32%,90.78%,86.46%,"3.167,24",1-Altamente propenso


In [168]:
#Tratando com separador de milhares como '.' e de decimal como ','
tratar_ponto_virgula=['VR_DESPESAS_SIN','VR_EM_ATRASO_COM_JUROS','VR_VALOR_COM_DESPESA']

for coluna in tratar_ponto_virgula:
  esc[coluna]=esc[coluna].apply(lambda x: str(x).replace('.',''))
  esc[coluna]=esc[coluna].apply(lambda x: str(x).replace(',','.'))
  esc[coluna]=esc[coluna]=esc[coluna]=esc[coluna].astype(float)
  print(esc[coluna])

0           0.00
1           0.00
2        1068.05
3           0.00
4           0.00
          ...   
46356     216.52
46357     150.30
46358       0.00
46359       0.00
46360       0.00
Name: VR_DESPESAS_SIN, Length: 46361, dtype: float64
0        14903.00
1         4500.86
2        44948.26
3         8351.37
4        17236.08
           ...   
46356     2326.04
46357     4064.40
46358     5128.30
46359     7198.24
46360     1239.13
Name: VR_EM_ATRASO_COM_JUROS, Length: 46361, dtype: float64
0         2845.63
1          822.46
2        10242.91
3         1519.35
4         3167.24
           ...   
46356     1746.72
46357     2820.98
46358     3368.98
46359     4731.00
46360      814.60
Name: VR_VALOR_COM_DESPESA, Length: 46361, dtype: float64


### Mesclando as duas tabelas

In [169]:
#Fazendo o merge das Tabelas
merged=esc.merge(concatenado,on='NR_RAS',how='inner')

In [170]:
#Eliminando colunas indesejadas
merged=merged[col_final]

###Tratando os números de telefone e email

In [171]:
#Validação de Celular
col_celular='NR_CELULAR_PORTO'
filtro=np.char.str_len(np.array(merged[col_celular]).astype(str))!=11
merged.loc[filtro,col_celular]=''

#Validação de Email
#Aplicando lower e strip na coluna de email
coluna_email='EMAIL_1'
merged[coluna_email]=merged[coluna_email].str.lower().str.strip()
#Apagando emails não validados
merged[coluna_email].loc[merged[coluna_email].str.find('@')<0]=''
merged['num @']=merged[coluna_email].str.len()-merged[coluna_email].str.replace('@','').str.len()
filtro=merged['num @']!=1
merged.loc[filtro,coluna_email]=''
merged.drop('num @',axis=1,inplace=True)

<ipython-input-171-4b88d990f8a8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged[coluna_email].loc[merged[coluna_email].str.find('@')<0]=''


###Tratar coluna de Nome

In [172]:
coluna_nomes=['NM_GARANTIDO']
for coluna_nome in coluna_nomes:
  merged[coluna_nome]=merged[coluna_nome].str.strip().str.title()

##Exportando

In [173]:
#Exportando Tabela Auxiliar para auxiliary
concatenado.to_csv(auxiliary+'Concatenado.csv',index=False)
#Exportando o arquivo para Output
merged.to_csv(csv_saida,index=False)

##Movendo arquivos

In [174]:
#Movendo arquivos para processedfiles
if mover:
  arquivos=[nome_esc,nome_tb]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)